In [129]:
# ────────────────────────────────────────────────
# 1. Cargar librerías necesarias
# ────────────────────────────────────────────────

import os
# Permite acceder a variables de entorno (como claves de API en un archivo .env)

import time
# Permite hacer pausas controladas (ej. time.sleep(1) = espera 1 segundo) para no saturar APIS.

import pandas as pd  
# Permite trabajar con tablas (como Excel, pero en código)

import spotipy  
# Librería oficial para conectarse a Spotify

from spotipy.oauth2 import SpotifyClientCredentials  
# OAuth es un protocolo de autorización seguro que permite a tu app acceder a Spotify usando credenciales de cliente (ID + secreto)

from dotenv import load_dotenv  
# → Herramienta para leer el archivo '.env' donde guardas tus claves secretas
# Así no las dejas expuestas en el código


In [56]:
pip show python-dotenv

Name: python-dotenv
Version: 1.2.1
Summary: Read key-value pairs from a .env file and set them as environment variables
Home-page: 
Author: 
Author-email: Saurabh Kumar <me+github@saurabh-kumar.com>
License: 
Location: c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: 
Required-by: webdriver-manager
Note: you may need to restart the kernel to use updated packages.


In [57]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [130]:
# ════════════════════════════════════════════════════════════════════════════
# 2. CARGAR Y VALIDAR CREDENCIALES DE SPOTIFY
# ════════════════════════════════════════════════════════════════════════════

# Carga las variables de entorno desde el archivo .env
# 'override=True' → Reemplaza valores existentes, útil al actualizar credenciales
load_dotenv('.env', override=True)

# Extrae las credenciales del entorno
id_spotify = os.getenv('SPOTIFY_CLIENT_ID')        # ID público de tu aplicación
secreto_spotify = os.getenv('SPOTIFY_CLIENT_SECRET')  # Clave secreta (¡nunca la compartas!)

# Validación de seguridad: Verifica que ambas credenciales existan
if not id_spotify or not secreto_spotify:
    raise ValueError(
        "Error: Faltan credenciales en el archivo .env"
        "   Asegúrate de tener:"
        "   - SPOTIFY_CLIENT_ID=tu_client_id"
        "   - SPOTIFY_CLIENT_SECRET=tu_client_secret"
    )

# Crea el objeto de autenticación con las credenciales validadas
mis_credenciales = SpotifyClientCredentials(
    client_id=id_spotify,
    client_secret=secreto_spotify
)

# Inicializa la conexión con la API de Spotify
spotify = spotipy.Spotify(auth_manager=mis_credenciales)
print("Conexión exitosa a la API de Spotify")

# ────────────────────────────────────────────────
# En este punto ya podemos usar spotify para hacer consultas:
# - spotify.search() → Buscar canciones, artistas, álbumes
# - spotify.artist() → Información de un artista
# - spotify.track() → Detalles de una canción
# ────────────────────────────────────────────────


Conexión exitosa a la API de Spotify


In [180]:
# ────────────────────────────────────────────────
# 3. PRUEBA DE CONEXIÓN CON SPOTIFY
# ────────────────────────────────────────────────
# Objetivo: verificar que la conexión funciona y que recibimos datos.
# Usamos una búsqueda VÁLIDA: "artist" por género (porque genre: solo funciona con artistas).

prueba = spotify.search(
    q='genre:latin',   # Buscamos artistas del género "latin"
    type='artist',     # Tipo de búsqueda: artistas
    limit=1            # Solo 1 resultado para la prueba
)

# Tomamos el primer artista
artista = prueba['artists']['items'][0]

# Mostramos su nombre para confirmar que recibimos datos
print("Conexión exitosa con Spotify")
print("Artista de prueba encontrado:", artista['name'])


Conexión exitosa con Spotify
Artista de prueba encontrado: Bad Bunny


In [178]:
# ────────────────────────────────────────────────
# 4. BÚSQUEDA DE ARTISTAS Y ÁLBUMES POR GÉNERO Y AÑO
# ────────────────────────────────────────────────

# Queremos canciones por género y año.
# Spotify no permite usar "genre:" y "year:" juntos en una sola búsqueda de canciones,
# así que dividimos el trabajo en pasos.

generos = ["flamenco", "latin", "jazz", "rock"]
año = 2016
todas_las_canciones = []
albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).

print("Buscando canciones del año", año)
print()

for genero in generos:
    print("Género:", genero)

    # 👉 ANOTACIÓN: "genre:" SOLO funciona al buscar artistas (type="artist")
    #    No funciona con álbumes ni canciones directamente.
    resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=10)
    artistas = resultado_artistas["artists"]["items"]

    for artista in artistas:
        nombre_artista = artista["name"]

        # 👉 ANOTACIÓN: "artist:" y "year:" SÍ se pueden usar juntos
        #    al buscar álbumes (type="album") o canciones (type="track").
        #    Aquí buscamos álbumes del artista en el año indicado.
        busqueda = "artist:" + nombre_artista + " year:" + str(año)
        resultado_albumes = spotify.search(q=busqueda, type="album", limit=10)
        albumes = resultado_albumes["albums"]["items"]

        for album in albumes:
            id_album = album["id"]
            nombre_album = album["name"]
            
            # Si ya vimos este álbum, saltamos al siguiente
            if id_album in albumes_ya_vistos:
                continue
            
            # Si no lo habíamos visto, lo marcamos como visto
            albumes_ya_vistos.add(id_album)

            # Pedimos las canciones del álbum (esto no usa anotaciones, es una llamada directa)
            resultado_canciones = spotify.album_tracks(id_album)
            canciones = resultado_canciones["items"]

            for cancion in canciones:
                info = {
                    "nombre": cancion["name"],
                    "artista": nombre_artista,
                    "album": nombre_album,
                    "genero": genero,
                    "año": año
                }
                todas_las_canciones.append(info)

    print("  → Canciones de este género añadidas")
    print()

# Mostramos un resumen de lo encontrado
print("RESUMEN:")

# Contamos las canciones de cada género una por una
for genero in generos:
    contador = 0
    for cancion in todas_las_canciones:
        if cancion["genero"] == genero:
            contador = contador + 1
    print(f"- {genero}: {contador} canciones")

print("Total de canciones encontradas:", len(todas_las_canciones))

Buscando canciones del año 2016

Género: flamenco
  → Canciones de este género añadidas

Género: latin
  → Canciones de este género añadidas

Género: jazz
  → Canciones de este género añadidas

Género: rock
  → Canciones de este género añadidas

RESUMEN:
- flamenco: 0 canciones
- latin: 217 canciones
- jazz: 460 canciones
- rock: 393 canciones
Total de canciones encontradas: 1070
